In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
class Perceptron(nn.Module):
    def __init__(self):
        super(Perceptron, self).__init__()        
        self.fc1 = nn.Linear(2, 1)
        
    def forward(self, x):
        x = self.fc1(x)
        return x

In [3]:
net = Perceptron()
print(net)

Perceptron(
  (fc1): Linear(in_features=2, out_features=1, bias=True)
)


In [4]:
and_inputs = [(0.0, 0.0), (0.0, 1.0), (1.0, 0.0), (1.0, 1.0)]
and_outputs = [(0.0,), (1.0,), (1.0,), (1.0,)]

In [5]:
optimizer = optim.Adam(net.parameters(), lr=0.01)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.01
    weight_decay: 0
)

In [6]:
mse_loss = nn.MSELoss()
mse_loss

MSELoss()

In [7]:
training_data = [(torch.Tensor(x), torch.Tensor(y)) for x, y in zip(and_inputs, and_outputs)]
training_data

[(tensor([0., 0.]), tensor([0.])),
 (tensor([0., 1.]), tensor([1.])),
 (tensor([1., 0.]), tensor([1.])),
 (tensor([1., 1.]), tensor([1.]))]

In [8]:
testing_data = [(torch.Tensor(x), torch.Tensor(y)) for x, y in zip(and_inputs, and_outputs)]
testing_data

[(tensor([0., 0.]), tensor([0.])),
 (tensor([0., 1.]), tensor([1.])),
 (tensor([1., 0.]), tensor([1.])),
 (tensor([1., 1.]), tensor([1.]))]

In [9]:
for epoch in range(60):
    running_loss = 0.0
    for data in training_data:
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = mse_loss(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    if (epoch + 1) % 10 == 0:
        print('Epoch [{0}] Loss: {1:.10f}'.format(epoch + 1, running_loss))

Epoch [10] Loss: 0.4330166227
Epoch [20] Loss: 0.3295403104
Epoch [30] Loss: 0.2914422434
Epoch [40] Loss: 0.2736582458
Epoch [50] Loss: 0.2641635016
Epoch [60] Loss: 0.2589218058


In [10]:
for param in net.parameters():
    print(param.data)

tensor([[0.4329, 0.4698]])
tensor([0.3108])


In [11]:
print("")
print("Final results:")
for data in testing_data:
    inputs, labels = data
    with torch.no_grad():
        output = net(inputs)
        print(inputs, output)


Final results:
tensor([0., 0.]) tensor([0.3108])
tensor([0., 1.]) tensor([0.7806])
tensor([1., 0.]) tensor([0.7437])
tensor([1., 1.]) tensor([1.2135])


In [12]:
weights = []
for param in net.parameters():
    weights.append(param.data.numpy().tolist())
weights

[[[0.43286392092704773, 0.4697785973548889]], [0.3108426630496979]]

In [13]:
def flat_weights(weights, flattned):
    for w in weights:
        if type(w) == list:
            flat_weights(w, flattned)
        else:
            flattned.append(w)

In [14]:
flattned_weights = []
flat_weights(weights, flattned_weights)
flattned_weights

[0.43286392092704773, 0.4697785973548889, 0.3108426630496979]

In [16]:
flattned_weights[2] = 0.29
flattned_weights

[0.43286392092704773, 0.4697785973548889, 0.29]

In [17]:
def search_and_replace(weights, replacements, replace_idx=0):
    for i, l in enumerate(weights):
        if type(l) == list:
            replace_idx = search_and_replace(l, replacements, replace_idx)
        else:
            weights[i] = replacements[replace_idx]
            replace_idx += 1  
    return replace_idx

In [19]:
_ = search_and_replace(weights, flattned_weights, 0)

In [20]:
weights

[[[0.43286392092704773, 0.4697785973548889]], [0.29]]

In [21]:
for param, new_weights in zip(net.parameters(), weights):
    param.data = torch.tensor(new_weights, requires_grad=True)

In [22]:
print("")
print("Final results:")
for data in testing_data:
    inputs, labels = data
    with torch.no_grad():
        output = net(inputs)
        print(inputs, output)


Final results:
tensor([0., 0.]) tensor([0.2900])
tensor([0., 1.]) tensor([0.7598])
tensor([1., 0.]) tensor([0.7229])
tensor([1., 1.]) tensor([1.1926])


In [23]:
for param in net.parameters():
    print(param.data)

tensor([[0.4329, 0.4698]])
tensor([0.2900])


In [24]:
class MyModule(nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()
        self.choices = nn.ModuleDict({
                'conv': nn.Conv2d(10, 10, 3),
                'pool': nn.MaxPool2d(3)
        })
        self.activations = nn.ModuleDict([
                ['lrelu', nn.LeakyReLU()],
                ['prelu', nn.PReLU()]
        ])

    def forward(self, x, choice, act):
        x = self.choices[choice](x)
        x = self.activations[act](x)
        return x